In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

# Hyper-parameters
num_epochs = 5 # do higher number for a better converg result
batch_size = 4
learning_rate = 0.001

# dataset has PILImage images of range[0, 1].
# We transform them to Tensors of normalized range [-1, 1]
transform = transforms.Compose(
  [transforms.ToTensor(),
  transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]
)

train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)

train_dataset = torchvision.datasets.CIFAR10(root='./data', 
    train=True, transform=transform, download=True)

test_dataset = torchvision.datasets.CIFAR10(root='./data', 
    train=False, transform=transform, download=True)

train_loader = torch.utils.data.DataLoader(dataset=train_dataset, 
    batch_size=batch_size, shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset, 
    batch_size=batch_size, shuffle=False)

classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

# implement conv net
class ConvNet(nn.Module):
  def __init__(self):
    super(ConvNet, self).__init__()
    self.conv1 = nn.Conv2d(3, 6, 5) # input_size, output_size, kernel_size (5x5 filter)
    self.pool = nn.MaxPool2d(2, 2)
    self.conv2 = nn.Conv2d(6, 16, 5)
    self.fc1 = nn.Linear(16*5*5, 120) # fully collected layer
    self.fc2 = nn.Linear(120, 84)
    self.fc3 = nn.Linear(84, 10)

  def forward(self, x):
    # 1. Convolutional and pooling layer
    x = self.conv1(x)
    x = F.relu(x) # relu (activation) does not change the dimension
    x = self.pool(x)

    # 2. Convolutional and pooling layer
    x = self.conv2(x)
    x = F.relu(x)
    x = self.pool(x)

    # 3. Flatten layer
    x = x.view(-1, 16*5*5)

    # 4. Fully connected layers
    x = self.fc1(x)
    x = F.relu(x)

    # 5. Fully connected layers
    x = self.fc2(x)
    x = F.relu(x)

    # 6. Fully connected layers
    x = self.fc3(x)

    # softmax is done by cross entropy loss 
    return x

model = ConvNet().to(device)

criterion = nn.CrossEntropyLoss() # includes softmax
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

n_total_steps = len(train_loader)
for epoch in range(num_epochs):
  for i, (images, labels) in enumerate(train_loader):
    # origin shape: [4, 3, 32, 32] = 4, 3, 1024
    # input_layer: 3 input channels, 6 output channels, 5 kernel size
    images = images.to(device)
    labels = labels.to(device)

    # Forward pass
    outputs = model(images)
    loss = criterion(outputs, labels)

    # Backward and optimize
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if (i+1) % 2000 == 0:
      print(f'Epoch [{epoch+1}/{num_epochs}], Step[{i+1}/{n_total_steps}], Loss: {loss.item():.4f}')

print('Finished Training')

with torch.no_grad():
  n_correct = 0
  n_samples = 0
  n_class_correct = [0 for i in range(10)]
  n_class_samples = [0 for i in range(10)]
  for images, labels in test_loader:
    images = images.to(device)
    labels = labels.to(device)
    outputs = model(images)
    # max returns (value, index)
    _, predicted = torch.max(outputs, 1)
    n_samples += labels.size(0)
    n_correct += (predicted == labels).sum().item()

    for i in range(batch_size):
      label = labels[i]
      pred = predicted[i]
      if (label == pred):
        n_class_correct[label] += 1
      n_class_samples[label] += 1

  acc = 100.0 * n_correct / n_samples
  print(f'accuracy = {acc}')

  for i in range(10):
    acc = 100.0 * n_class_correct[i] / n_class_samples[i]
    print(f'Accuracy of {classes[i]}: {acc}%')

cuda
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Epoch [1/5], Step[2000/12500], Loss: 2.2905
Epoch [1/5], Step[4000/12500], Loss: 2.3152
Epoch [1/5], Step[6000/12500], Loss: 2.2991
Epoch [1/5], Step[8000/12500], Loss: 2.3104
Epoch [1/5], Step[10000/12500], Loss: 2.2876
Epoch [1/5], Step[12000/12500], Loss: 2.2905
Epoch [2/5], Step[2000/12500], Loss: 2.0024
Epoch [2/5], Step[4000/12500], Loss: 2.3452
Epoch [2/5], Step[6000/12500], Loss: 2.6314
Epoch [2/5], Step[8000/12500], Loss: 1.8563
Epoch [2/5], Step[10000/12500], Loss: 1.7653
Epoch [2/5], Step[12000/12500], Loss: 1.4000
Epoch [3/5], Step[2000/12500], Loss: 1.8265
Epoch [3/5], Step[4000/12500], Loss: 1.6596
Epoch [3/5], Step[6000/12500], Loss: 1.7554
Epoch [3/5], Step[8000/12500], Loss: 1.9537
Epoch [3/5], Step[10000/12500], Loss: 1.6796
Epoch [3/5], Step[12000/12500], Loss: 1.7246
Epoch [4/5], Step[2000/12500], Loss: 1.1823
Epoch [4/5], Step[4000/12500], Loss: 1.